<a href="https://colab.research.google.com/github/tanaymukherjee/Deep-Learning-with-PyTorch/blob/master/03_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression with PyTorch

## Import packages and libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## Hyper-parameters 

In [2]:
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

## MNIST dataset (images and labels)

In [3]:
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


## Data loader (input pipeline)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Logistic regression model

In [5]:
model = nn.Linear(input_size, num_classes)

## Loss and optimizer

In [6]:
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

## Train the model

In [7]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2334
Epoch [1/5], Step [200/600], Loss: 2.1098
Epoch [1/5], Step [300/600], Loss: 2.0136
Epoch [1/5], Step [400/600], Loss: 1.9297
Epoch [1/5], Step [500/600], Loss: 1.8658
Epoch [1/5], Step [600/600], Loss: 1.7388
Epoch [2/5], Step [100/600], Loss: 1.7687
Epoch [2/5], Step [200/600], Loss: 1.6177
Epoch [2/5], Step [300/600], Loss: 1.6115
Epoch [2/5], Step [400/600], Loss: 1.5299
Epoch [2/5], Step [500/600], Loss: 1.4668
Epoch [2/5], Step [600/600], Loss: 1.5192
Epoch [3/5], Step [100/600], Loss: 1.4268
Epoch [3/5], Step [200/600], Loss: 1.3389
Epoch [3/5], Step [300/600], Loss: 1.3127
Epoch [3/5], Step [400/600], Loss: 1.3649
Epoch [3/5], Step [500/600], Loss: 1.2402
Epoch [3/5], Step [600/600], Loss: 1.3358
Epoch [4/5], Step [100/600], Loss: 1.1538
Epoch [4/5], Step [200/600], Loss: 1.1589
Epoch [4/5], Step [300/600], Loss: 1.2211
Epoch [4/5], Step [400/600], Loss: 1.1460
Epoch [4/5], Step [500/600], Loss: 1.1446
Epoch [4/5], Step [600/600], Loss:

## Test the model

In [8]:
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 83 %


/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


## Save the model checkpoint

In [9]:
torch.save(model.state_dict(), 'model.ckpt')